In [1]:
from google.colab import drive    #connecting with google drive where the cleaned data is present
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
!ls "/content/drive/MyDrive/ncd_bhutan_project/data/processed"  # this list all files in cleaned data


cleaned_data.csv


In [4]:
import pandas as pd   # loading data of cleaned data

# Load the cleaned CSV
df = pd.read_csv("/content/drive/MyDrive/ncd_bhutan_project/data/processed/cleaned_data.csv")
df.head()


,COUNTRY_DISPLAY,YEAR_DISPLAY,STARTYEAR,ENDYEAR,GHO_DISPLAY,DIMENSION_NAME,Value_num
0,Bhutan,1991,1991,1991,"Mean Non-HDL cholesterol, age-standardized",Female,3.00
1,Bhutan,1989,1989,1989,"Alcohol, recorded per capita (15+) consumption...",Beer,0.31
2,Bhutan,2020,2020,2020,"Alcohol, recorded per capita (15+) consumption...",NaN,2.70
3,Bhutan,2010,2010,2010,Prevalence of overweight among children and ad...,Male,9.00
4,Bhutan,2012,2012,2012,Premature deaths due to noncommunicable diseas...,Male,49.00


Advanced data cleaning starts from here

In [5]:
# Fill numeric missing values with mean          ## handle missing value
num_cols = df.select_dtypes(include=['float64','int64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

# Fill categorical missing values with mode
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])

df.isnull().sum()


,0
COUNTRY_DISPLAY,0
YEAR_DISPLAY,0
STARTYEAR,0
ENDYEAR,0
GHO_DISPLAY,0
DIMENSION_NAME,0
Value_num,0


In [7]:
# Convert columns to numeric where needed  ## fix data types
df['STARTYEAR'] = pd.to_numeric(df['STARTYEAR'], errors='coerce')
df['ENDYEAR'] = pd.to_numeric(df['ENDYEAR'], errors='coerce')
df['Value_num'] = pd.to_numeric(df['Value_num'], errors='coerce')

df.dtypes



,0
COUNTRY_DISPLAY,object
YEAR_DISPLAY,int64
STARTYEAR,int64
ENDYEAR,int64
GHO_DISPLAY,object
DIMENSION_NAME,object
Value_num,float64


In [8]:
before = len(df)   ## remove duplicates
df = df.drop_duplicates()
after = len(df)

print("Duplicates removed:", before - after)


Duplicates removed: 50


In [9]:
# Duration of record    ## creating new features
df["DURATION"] = df["ENDYEAR"] - df["STARTYEAR"]

# Risk level based on Value_num
df["RISK_LEVEL"] = pd.cut(
    df["Value_num"],
    bins=[0, 10, 20, 100],
    labels=["Low", "Medium", "High"]
)

# Normalized value for Value_num
df["VALUE_NORMALIZED"] = (df["Value_num"] - df["Value_num"].min()) / (df["Value_num"].max() - df["Value_num"].min())

df.head()


/tmp/ipython-input-3861194475.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DURATION"] = df["ENDYEAR"] - df["STARTYEAR"]
/tmp/ipython-input-3861194475.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["RISK_LEVEL"] = pd.cut(
/tmp/ipython-input-3861194475.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

,COUNTRY_DISPLAY,YEAR_DISPLAY,STARTYEAR,ENDYEAR,GHO_DISPLAY,DIMENSION_NAME,Value_num,DURATION,RISK_LEVEL,VALUE_NORMALIZED
0,Bhutan,1991,1991,1991,"Mean Non-HDL cholesterol, age-standardized",Female,3.00,0,Low,0.001042
1,Bhutan,1989,1989,1989,"Alcohol, recorded per capita (15+) consumption...",Beer,0.31,0,Low,0.000108
2,Bhutan,2020,2020,2020,"Alcohol, recorded per capita (15+) consumption...",Both sexes,2.70,0,Low,0.000938
3,Bhutan,2010,2010,2010,Prevalence of overweight among children and ad...,Male,9.00,0,Low,0.003125
4,Bhutan,2012,2012,2012,Premature deaths due to noncommunicable diseas...,Male,49.00,0,High,0.017014


In [10]:
df.to_csv("/content/drive/MyDrive/ncd_bhutan_project/data/processed/advanced_cleaned_ncd.csv", index=False)  ## saving final dataset
print("Saved as advanced_cleaned_ncd.csv")


Saved as advanced_cleaned_ncd.csv


In [11]:
df.info()  ## checking data
df.describe()


<class 'pandas.core.frame.DataFrame'>
Index: 3664 entries, 0 to 3713
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   COUNTRY_DISPLAY   3664 non-null   object  
 1   YEAR_DISPLAY      3664 non-null   int64   
 2   STARTYEAR         3664 non-null   int64   
 3   ENDYEAR           3664 non-null   int64   
 4   GHO_DISPLAY       3664 non-null   object  
 5   DIMENSION_NAME    3664 non-null   object  
 6   Value_num         3664 non-null   float64 
 7   DURATION          3664 non-null   int64   
 8   RISK_LEVEL        3487 non-null   category
 9   VALUE_NORMALIZED  3664 non-null   float64 
dtypes: category(1), float64(2), int64(4), object(3)
memory usage: 290.0+ KB


,YEAR_DISPLAY,STARTYEAR,ENDYEAR,Value_num,DURATION,VALUE_NORMALIZED
count,3664.000000,3664.000000,3664.000000,3664.000000,3664.0,3664.000000
mean,2005.018013,2005.018013,2005.018013,38.610186,0.0,0.013406
std,11.713954,11.713954,11.713954,224.698649,0.0,0.078020
min,1961.000000,1961.000000,1961.000000,0.000000,0.0,0.000000
25%,1997.000000,1997.000000,1997.000000,3.100000,0.0,0.001076
50%,2006.000000,2006.000000,2006.000000,6.600000,0.0,0.002292
75%,2014.000000,2014.000000,2014.000000,14.000000,0.0,0.004861
max,2030.000000,2030.000000,2030.000000,2880.000000,0.0,1.000000
